Notebook is based on m5-baseline (harupy), which is itself based on Very fst Model (ragnar123).

https://www.kaggle.com/ragnar123/very-fst-model<br>
https://www.kaggle.com/harupy/m5-baseline

# Imports

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

ROOT = pathlib.Path().absolute().parent
RAW_DATA_PATH = ROOT / 'data' / 'raw'
PROCESSED_DATA_PATH = ROOT / 'data' / 'processed'

DAYS_PRED = 28

# endure this project is in the path
sys.path.insert(0, ROOT.absolute().as_posix())

from src.data.process_data import reduce_memory_usage

In [4]:
df = \
(pd.read_parquet(PROCESSED_DATA_PATH / 'combined_dataset.parquet')
.astype({c:'category' for c in ['wday','month','year']}))

df

,part,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,...,event_name_ramadan starts,event_name_stpatricksday,event_name_superbowl,event_name_thanksgiving,event_name_valentinesday,event_name_veteransday,is_weekend,event,snap,sell_price_cent
0,train,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,...,False,False,False,False,False,False,True,False,False,46
1,train,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,...,False,False,False,False,False,False,True,False,False,156
2,train,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,...,False,False,False,False,False,False,True,False,False,317
3,train,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,...,False,False,False,False,False,False,True,False,False,598
4,train,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,2011-01-29,...,False,False,False,False,False,False,True,False,False,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47735392,evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,...,False,False,False,False,False,False,True,True,False,298
47735393,evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,...,False,False,False,False,False,False,True,True,False,248
47735394,evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,...,False,False,False,False,False,False,True,True,False,398
47735395,evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,...,False,False,False,False,False,False,True,True,False,128
